In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import altair as alt
from altair_saver import save

In [2]:
#we read the table with the results of the 1st experiment
df = pd.read_excel("Experimental analysis 1.xlsx")


In [3]:
conditions = [
    df['problem'].str.contains('euclides'),
    df['problem'].str.contains('eratostenes'),
    df['problem'].str.contains('knapsack')
]

values = ['Euclid', 'Eratosthenes', 'Knapsack']

df.loc[:, 'root_problem'] = np.select(conditions, values, default="Binary Search")
df.columns



Index(['rute', 'model', 'problem', 'iteration',
       'Detecta de que problema se trata?',
       'Da un código funcional en python?', 'Explica el código?',
       'Utiliza un razonamiento correcto?', 'Da un código recursivo?',
       'Hace los pasos que se le pide en la descripción?',
       'Hace los pasos que se le pide en la descripción? (Explicación)',
       'comentarios de razonamientos incorrectos', 'root_problem'],
      dtype='object')

In [4]:
def dataframe_totales(df, columna_agrupar):
    df_total = df.groupby(columna_agrupar).size().rename('Total')
    df_new = df.groupby(columna_agrupar).sum()
    df_new = df_new.join(df_total)
    return df_new

def dataframe_porcentajes (df, columna_agrupar):
    df_total = dataframe_totales(df, columna_agrupar)

    for col in df_total.columns[:-1]: 
        df_total[col] = ((df_total[col] / df_total['Total']) * 100).round(1)
    df_total.drop(columns=['Total'], inplace=True)
    
    return df_total




In [5]:
df_select = df.iloc[:,[1, 2,4,5,6,7,8,9,12]]

#Consideramos los problemas de los prompts 1 y 2
filtered_no_NL_df = df_select[~df_select['problem'].str.contains('_NL')]
filtered_NL_df = df_select[df_select['problem'].str.contains('_NL')]

new_problem_names = {
    'busqueda_binaria': 'Binary Search',
    'busqueda_binaria_rec': 'Binary Search Recursive',
    'eratostenes': 'Eratosthenes',
    'eratostenes_rec': 'Eratosthenes Recursive',
    'euclides': 'Euclid',
    'euclides_rec': 'Euclid Recursive',
    'knapsack': 'Knapsack',
    'knapsack_rec': 'Knapsack Recursive',
}

filtered_no_NL_df['problem'] = filtered_no_NL_df['problem'].map(new_problem_names)


C:\Users\34649\AppData\Local\Temp\ipykernel_64504\1280389070.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_no_NL_df['problem'] = filtered_no_NL_df['problem'].map(new_problem_names)


In [6]:
# Gives a functional code

df_NL_model_resumen_1 = dataframe_porcentajes (filtered_NL_df.iloc[:,[0,3]],'model')
df_no_NL_model_resumen_1 =  dataframe_porcentajes (filtered_no_NL_df.iloc[:,[0,3]],'model')

# Performs the steps requested in the description


df_NL_model_resumen_2 = dataframe_porcentajes (filtered_NL_df.iloc[:,[0,7]],'model')
df_no_NL_model_resumen_2 =  dataframe_porcentajes (filtered_no_NL_df.iloc[:,[0,7]],'model')

In [7]:
df_NL_model_resumen_1

,Da un código funcional en python?
model,
codellama,100.0
deepseek,75.0
gpt-3.5,91.7
gpt-4,100.0
llama2,75.0
llama3,100.0
platypus,25.0
qwen,50.0


In [8]:
df_NL_model_resumen_2

,Hace los pasos que se le pide en la descripción?
model,
codellama,100.0
deepseek,75.0
gpt-3.5,91.7
gpt-4,100.0
llama2,50.0
llama3,100.0
platypus,25.0
qwen,50.0


In [9]:
df_no_NL_model_resumen_1

,Da un código funcional en python?
model,
codellama,87.5
deepseek,75.0
gpt-3.5,91.7
gpt-4,100.0
llama2,25.0
llama3,100.0
platypus,62.5
qwen,87.5


In [10]:
df_no_NL_model_resumen_2

,Hace los pasos que se le pide en la descripción?
model,
codellama,37.5
deepseek,37.5
gpt-3.5,87.5
gpt-4,75.0
llama2,12.5
llama3,75.0
platypus,50.0
qwen,75.0


In [11]:
#Strategy 1 % by model and problem that gives a functional code

df_total_no_NL = filtered_no_NL_df.groupby(['model', 'problem']).size().rename('Total')
df_no_NL_3 = filtered_no_NL_df[filtered_no_NL_df.iloc[:, 3] == True]
grouped_df_no_NL_3 = df_no_NL_3.groupby(['model', 'problem']).size().reset_index(name='Count')
df_new_no_NL_3 = pd.merge(grouped_df_no_NL_3, df_total_no_NL, on=['model', 'problem'])
df_new_no_NL_3['percentage'] = (df_new_no_NL_3['Count'] / df_new_no_NL_3['Total'] * 100).round(1)

print(df_new_no_NL_3)


        model                  problem  Count  Total  percentage
0   codellama            Binary Search      3      3       100.0
1   codellama             Eratosthenes      3      3       100.0
2   codellama   Eratosthenes Recursive      3      3       100.0
3   codellama                   Euclid      3      3       100.0
4   codellama         Euclid Recursive      3      3       100.0
5   codellama                 Knapsack      3      3       100.0
6   codellama       Knapsack Recursive      3      3       100.0
7    deepseek            Binary Search      3      3       100.0
8    deepseek  Binary Search Recursive      3      3       100.0
9    deepseek             Eratosthenes      3      3       100.0
10   deepseek   Eratosthenes Recursive      3      3       100.0
11   deepseek                   Euclid      3      3       100.0
12   deepseek         Euclid Recursive      3      3       100.0
13    gpt-3.5            Binary Search      3      3       100.0
14    gpt-3.5  Binary Sea

In [12]:
df = df_new_no_NL_3

custom_order = ['llama3', 'gpt-4','gpt-3.5', 'codellama', 'deepseek', 'llama2', 'platypus', 'qwen']
order_map = {model: idx for idx, model in enumerate(custom_order)}


color_scale = alt.Scale(domain=[1, 2, 3],
                        range=['orange', 'yellow', 'green'])

df['model_order'] = df['model'].map(order_map)

chart = alt.Chart(df).mark_rect(stroke='gray', strokeWidth=1).encode(
    x=alt.X('problem:N', title='Problem', sort='ascending', axis=alt.Axis(labelAngle=-25, labelFontSize=16, titleFontSize=16)),
    y=alt.Y('model:N', title='Model', sort=alt.EncodingSortField(field='model_order', order='ascending'),
            axis=alt.Axis(grid=False, labelFontSize=16, titleFontSize=18)), 
    fill=alt.Fill('Count:Q', scale=color_scale),
    tooltip=['model', 'problem', 'Count']
).properties(
    width=600,
    height=400,
    title=alt.TitleParams('Model vs. Problem', fontSize=20)
).configure_legend(
    disable=True
)

output_html = 'Estrategia 1 Problemas que generan un codigo funcional.html'
chart.save(output_html)

print(f'Gráfico guardado como "{output_html}"')

Gráfico guardado como "Estrategia 1 Problemas que generan un codigo funcional.html"


C:\Users\34649\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


In [13]:
#Strategy 2 % by model and problem gives a funcional code

df_total_NL = filtered_NL_df.groupby(['model', 'root_problem']).size().rename('Total')
df_NL_3 = filtered_NL_df[filtered_NL_df.iloc[:, 3] == True]
grouped_df_NL_3 = df_NL_3.groupby(['model', 'root_problem']).size().reset_index(name='Count')
df_new_NL_3 = pd.merge(grouped_df_NL_3, df_total_NL, on=['model', 'root_problem'])
df_new_NL_3['percentage'] = (df_new_NL_3['Count'] / df_new_NL_3['Total'] * 100).round(1)

print(df_new_NL_3)

        model   root_problem  Count  Total  percentage
0   codellama  Binary Search      3      3       100.0
1   codellama   Eratosthenes      3      3       100.0
2   codellama         Euclid      3      3       100.0
3   codellama       Knapsack      3      3       100.0
4    deepseek   Eratosthenes      3      3       100.0
5    deepseek         Euclid      3      3       100.0
6    deepseek       Knapsack      3      3       100.0
7     gpt-3.5  Binary Search      2      3        66.7
8     gpt-3.5   Eratosthenes      3      3       100.0
9     gpt-3.5         Euclid      3      3       100.0
10    gpt-3.5       Knapsack      3      3       100.0
11      gpt-4  Binary Search      3      3       100.0
12      gpt-4   Eratosthenes      3      3       100.0
13      gpt-4         Euclid      3      3       100.0
14      gpt-4       Knapsack      3      3       100.0
15     llama2   Eratosthenes      3      3       100.0
16     llama2         Euclid      3      3       100.0
17     lla

In [14]:
df = df_new_NL_3

custom_order = ['llama3', 'gpt-4','gpt-3.5', 'codellama', 'deepseek', 'llama2', 'platypus', 'qwen']
order_map = {model: idx for idx, model in enumerate(custom_order)}

color_scale = alt.Scale(domain=[1, 2, 3],
                        range=['orange', 'yellow', 'green'])
df['model_order'] = df['model'].map(order_map)

chart = alt.Chart(df).mark_rect(stroke='gray', strokeWidth=1).encode(
    x=alt.X('root_problem:N', title='Problem', sort='ascending', axis=alt.Axis(labelAngle=-25, labelFontSize=16, titleFontSize=16)),
    y=alt.Y('model:N', title='Model', sort=alt.EncodingSortField(field='model_order', order='ascending'),
            axis=alt.Axis(grid=False, labelFontSize=16, titleFontSize=18)), 
    fill=alt.Fill('Count:Q', scale=color_scale),
    tooltip=['model', 'root_problem', 'Count']
).properties(
    width=600,
    height=400,
    title=alt.TitleParams('Model vs. Problem', fontSize=20)
).configure_legend(
    disable=True
)



output_html = 'Estrategia 2 Problemas que generan un codigo funcional.html'
chart.save(output_html)


print(f'Gráfico guardado como "{output_html}"')

Gráfico guardado como "Estrategia 2 Problemas que generan un codigo funcional.html"


C:\Users\34649\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


In [15]:
#Strategy 1 % by model and problem follow the steps

df_total_no_NL = filtered_no_NL_df.groupby(['model', 'problem']).size().rename('Total')
df_no_NL_4 = filtered_no_NL_df[filtered_no_NL_df.iloc[:, 7] == True]
grouped_df_no_NL_4 = df_no_NL_4.groupby(['model', 'problem']).size().reset_index(name='Count')
df_new_no_NL_4 = pd.merge(grouped_df_no_NL_4, df_total_no_NL, on=['model', 'problem'])
df_new_no_NL_4['percentage'] = (df_new_no_NL_4['Count'] / df_new_no_NL_4['Total'] * 100).round(1)

print(df_new_no_NL_4)


        model                  problem  Count  Total  percentage
0   codellama            Binary Search      3      3       100.0
1   codellama             Eratosthenes      3      3       100.0
2   codellama                 Knapsack      3      3       100.0
3    deepseek            Binary Search      3      3       100.0
4    deepseek  Binary Search Recursive      3      3       100.0
5    deepseek             Eratosthenes      3      3       100.0
6     gpt-3.5            Binary Search      3      3       100.0
7     gpt-3.5  Binary Search Recursive      3      3       100.0
8     gpt-3.5             Eratosthenes      3      3       100.0
9     gpt-3.5   Eratosthenes Recursive      1      3        33.3
10    gpt-3.5                   Euclid      2      3        66.7
11    gpt-3.5         Euclid Recursive      3      3       100.0
12    gpt-3.5                 Knapsack      3      3       100.0
13    gpt-3.5       Knapsack Recursive      3      3       100.0
14      gpt-4            

In [16]:
df = df_new_no_NL_4

custom_order = ['llama3', 'gpt-4','gpt-3.5', 'codellama', 'deepseek', 'llama2', 'platypus', 'qwen']

order_map = {model: idx for idx, model in enumerate(custom_order)}

color_scale = alt.Scale(domain=[1, 2, 3],
                        range=['orange', 'yellow', 'green'])
df['model_order'] = df['model'].map(order_map)

chart = alt.Chart(df).mark_rect(stroke='gray', strokeWidth=1).encode(
    x=alt.X('problem:N', title='Problem', sort='ascending', axis=alt.Axis(labelAngle=-25, labelFontSize=16, titleFontSize=16)),
    y=alt.Y('model:N', title='Model', sort=alt.EncodingSortField(field='model_order', order='ascending'),
            axis=alt.Axis(grid=False, labelFontSize=16, titleFontSize=18)), 
    fill=alt.Fill('Count:Q', scale=color_scale),
    tooltip=['model', 'problem', 'Count']
).properties(
    width=600,
    height=400,
    title=alt.TitleParams('Model vs. Problem', fontSize=20)
).configure_legend(
    disable=True
)


output_html = 'Estrategia 1 Problemas que siguen los pasos.html'
chart.save(output_html)

print(f'Gráfico guardado como "{output_html}"')

Gráfico guardado como "Estrategia 1 Problemas que siguen los pasos.html"


C:\Users\34649\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


In [17]:
#Strategy 2 % by model and problem following the steps
df_total_NL = filtered_NL_df.groupby(['model', 'root_problem']).size().rename('Total')
df_NL_4 = filtered_NL_df[filtered_NL_df.iloc[:, 7] == True]
grouped_df_NL_4 = df_NL_4.groupby(['model', 'root_problem']).size().reset_index(name='Count')
df_new_NL_4 = pd.merge(grouped_df_NL_4, df_total_NL, on=['model', 'root_problem'])
df_new_NL_4['percentage'] = (df_new_NL_4['Count'] / df_new_NL_4['Total'] * 100).round(1)

print(df_new_NL_4)


        model   root_problem  Count  Total  percentage
0   codellama  Binary Search      3      3       100.0
1   codellama   Eratosthenes      3      3       100.0
2   codellama         Euclid      3      3       100.0
3   codellama       Knapsack      3      3       100.0
4    deepseek   Eratosthenes      3      3       100.0
5    deepseek         Euclid      3      3       100.0
6    deepseek       Knapsack      3      3       100.0
7     gpt-3.5  Binary Search      2      3        66.7
8     gpt-3.5   Eratosthenes      3      3       100.0
9     gpt-3.5         Euclid      3      3       100.0
10    gpt-3.5       Knapsack      3      3       100.0
11      gpt-4  Binary Search      3      3       100.0
12      gpt-4   Eratosthenes      3      3       100.0
13      gpt-4         Euclid      3      3       100.0
14      gpt-4       Knapsack      3      3       100.0
15     llama2   Eratosthenes      3      3       100.0
16     llama2       Knapsack      3      3       100.0
17     lla

In [18]:
df = df_new_NL_4

custom_order = ['llama3', 'gpt-4','gpt-3.5', 'codellama', 'deepseek', 'llama2', 'platypus', 'qwen']
order_map = {model: idx for idx, model in enumerate(custom_order)}

color_scale = alt.Scale(domain=[1, 2, 3],
                        range=['orange', 'yellow', 'green'])
df['model_order'] = df['model'].map(order_map)

chart = alt.Chart(df).mark_rect(stroke='gray', strokeWidth=1).encode(
    x=alt.X('root_problem:N', title='Problem', sort='ascending', axis=alt.Axis(labelAngle=-25, labelFontSize=16, titleFontSize=16)),
    y=alt.Y('model:N', title='Model', sort=alt.EncodingSortField(field='model_order', order='ascending'),
            axis=alt.Axis(grid=False, labelFontSize=16, titleFontSize=18)), 
    fill=alt.Fill('Count:Q', scale=color_scale),
    tooltip=['model', 'root_problem', 'Count']
).properties(
    width=600,
    height=400,
    title=alt.TitleParams('Model vs. Problem', fontSize=20)
).configure_legend(
    disable=True
)


output_html = 'Estrategia 2 Problemas que siguen los pasos.html'
chart.save(output_html)

print(f'Gráfico guardado como "{output_html}"')

Gráfico guardado como "Estrategia 2 Problemas que siguen los pasos.html"


C:\Users\34649\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


In [19]:
#We want to analyse when it is not directly requested that the code is recursive what percentage of the answers give a recursive code.filtered_strategy_1_no_rec = filtered_no_NL_df[~filtered_no_NL_df['problem'].str.contains('Recursive')]
filtered_strategy_1_no_rec = filtered_no_NL_df[~filtered_no_NL_df['problem'].str.contains('Recursive')]
filtered_strategy_1_no_rec_total = filtered_strategy_1_no_rec.groupby(['model', 'root_problem']).size().reset_index(name='Total')
df__strategy_1_no_rec_5 = filtered_strategy_1_no_rec[filtered_strategy_1_no_rec.iloc[:, 6] == True] #codigos recursivos

grouped_strategy_1_no_rec_5 = df__strategy_1_no_rec_5.groupby(['model', 'root_problem']).size().reset_index(name='Count')

df_new_5 = pd.merge(grouped_strategy_1_no_rec_5, filtered_strategy_1_no_rec_total, on=['model', 'root_problem'])
df_new_5['percentage'] = (df_new_5['Count'] / df_new_5['Total'] * 100).round(1)


print(df_new_5)




      model   root_problem  Count  Total  percentage
0  deepseek  Binary Search      3      3       100.0


In [20]:
filtered_strategy_1_rec = filtered_no_NL_df[filtered_no_NL_df['problem'].str.contains('Recursive')]
filtered_strategy_1_rec_total = filtered_strategy_1_rec.groupby(['model', 'problem']).size().reset_index(name='Total')
df__strategy_1_rec_5 = filtered_strategy_1_rec[filtered_strategy_1_rec.iloc[:, 6] == True] #codigos recursivos

grouped_strategy_1_rec_5 = df__strategy_1_rec_5.groupby(['model', 'problem']).size().reset_index(name='Count')

df_new_5 = pd.merge(grouped_strategy_1_rec_5, filtered_strategy_1_rec_total, on=['model', 'problem'])
df_new_5['percentage'] = (df_new_5['Count'] / df_new_5['Total'] * 100).round(1)

print(df_new_5)


        model                  problem  Count  Total  percentage
0   codellama  Binary Search Recursive      3      3       100.0
1   codellama   Eratosthenes Recursive      3      3       100.0
2   codellama         Euclid Recursive      3      3       100.0
3   codellama       Knapsack Recursive      3      3       100.0
4    deepseek  Binary Search Recursive      3      3       100.0
5    deepseek         Euclid Recursive      3      3       100.0
6    deepseek       Knapsack Recursive      3      3       100.0
7     gpt-3.5  Binary Search Recursive      3      3       100.0
8     gpt-3.5   Eratosthenes Recursive      1      3        33.3
9     gpt-3.5         Euclid Recursive      3      3       100.0
10    gpt-3.5       Knapsack Recursive      3      3       100.0
11      gpt-4  Binary Search Recursive      3      3       100.0
12      gpt-4   Eratosthenes Recursive      2      3        66.7
13      gpt-4         Euclid Recursive      3      3       100.0
14      gpt-4       Knaps

In [21]:
df = df_new_5
custom_order = ['llama3', 'gpt-4','gpt-3.5', 'codellama', 'deepseek', 'llama2', 'platypus', 'qwen']
order_map = {model: idx for idx, model in enumerate(custom_order)}

color_scale = alt.Scale(domain=[1, 2, 3],
                        range=['orange', 'yellow', 'green'])
df['model_order'] = df['model'].map(order_map)

chart = alt.Chart(df).mark_rect(stroke='gray', strokeWidth=1).encode(
    x=alt.X('problem:N', title='Problem', sort='ascending', axis=alt.Axis(labelAngle=-25, labelFontSize=16, titleFontSize=16)),
    y=alt.Y('model:N', title='Model', sort=alt.EncodingSortField(field='model_order', order='ascending'),
            axis=alt.Axis(grid=False, labelFontSize=16, titleFontSize=18)), 
    fill=alt.Fill('Count:Q', scale=color_scale),
    tooltip=['model', 'problem', 'Count']
).properties(
    width=600,
    height=400,
    title=alt.TitleParams('Model vs. Problem', fontSize=20)
).configure_legend(
    disable=True
)


output_html = 'Estrategia 1 Problemas que generan un código recursivo.html'
chart.save(output_html)

print(f'Gráfico guardado como "{output_html}"')

Gráfico guardado como "Estrategia 1 Problemas que generan un código recursivo.html"


C:\Users\34649\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


In [22]:
# Explain the code

df_resumen_coments = dataframe_porcentajes (df_select.iloc[:,[0,4]],'model')

# provide a correct reasoning

df_resumen_raz = dataframe_porcentajes (df_select.iloc[:,[0,5]],'model')

In [23]:
df_resumen_coments

,Explica el código?
model,
codellama,83.3
deepseek,41.7
gpt-3.5,8.3
gpt-4,94.4
llama2,33.3
llama3,75.0
platypus,0.0
qwen,66.7


In [24]:
df_resumen_raz

,Utiliza un razonamiento correcto?
model,
codellama,55.6
deepseek,41.7
gpt-3.5,0.0
gpt-4,72.2
llama2,8.3
llama3,66.7
platypus,0.0
qwen,50.0
